In [1]:
import numpy as np
import pandas as pd

In [2]:
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 KB 1.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 KB 3.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 4.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
from openai import OpenAI
import os

client = OpenAI(
    api_key = 'Your API key here'
)

def chatWithGPT(prompt,role="user",temperature=0):
  messages=[
    {
      "role": role, 
      "content": prompt
    }
  ]
  model="gpt-3.5-turbo"
  # model="gpt-4"

  response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=temperature,
    max_tokens=1024
  )
  return response.choices[0].message.content

In [7]:
df = pd.read_csv('data/SUBJ/test.csv')
df.head()
print(df['label'].value_counts())

label
1    500
0    500
Name: count, dtype: int64


In [9]:
import time

TP = 0
FP = 0
TN = 0
FN = 0
sentence_counter = 0
counter = 0

k = 1
MIN = (k-1) * 100
MAX = k * 100
for index, row in df.iterrows():
    if index >= MIN and index <MAX:
        sentence_counter += 1
   
        text = row['text']
        label = row['label']
   

        prompt = f"""You are an expert linguist. You should decide about a sentence whether is a subjective or an objective sentence.\
        A sentence is subjective if its content is based on or influenced by personal feelings, tastes, or opinions. Otherwise,
        the sentence is objective.\
        More precisely, a sentence is subjective if one or more of the following conditions apply:\
        1. expresses an explicit personal opinion from the author (e.g., speculations to draw conclusions);\
        2. includes sarcastic or ironic expressions;\
        3. gives exhortations of personal auspices;\
        4. contains discriminating or downgrading expressions;\
        5. contains rhetorical figures that convey the author’s opinion.\
        
        Here are some examples:\
        Q: Is the following sentence SUBJECTIVE or OBJECTIVE? SENTENCE: "she's not yet an actress , not quite a singer . . ."\
        A: SUBJECTIVE\
        Q: Is the following sentence SUBJECTIVE or OBJECTIVE? SENTENCE: "when she gets into trouble with the police simon represses his death wish and decides to help her out ."
        A: OBJECTIVE\
        
        Please classify the following text {text} in one of the two classes: (SUBJECTIVE, OBJECTIVE).\
        Please, answer with a single word: OBJECTIVE or SUBJECTIVE"""

        result =chatWithGPT(prompt)

        result = result.upper()
        print(label, '-->', result)
        if (label == 1 and result == 'SUBJECTIVE') or (label == 0 and result == 'OBJECTIVE'):
            counter += 1
        if (label == 0 and result == 'OBJECTIVE'):
            TN = TN +1
        if (label == 0 and result == 'SUBJECTIVE'):
            FP = FP +1
        if (label == 1 and result == 'SUBJECTIVE'):
            TP = TP +1
        if (label == 1 and result == 'OBJECTIVE'):
            FN = FN +1
        print('counter/sentence_counter:', counter, '/', sentence_counter)
        time.sleep(1)
        # if sentence_counter == 1:
        #     break

print('Accuracy:', (TP+TN)/(TP+FP+TN+FN))
print('Precision: ', TP/(TP+FP))
print('Recall: ', TP/(TP+FN))
print('F1:', TP/(TP+0.5*(FN+FP)))
print('TN:', TN)
print('FP:', FP)
print('FN:', FN)
print('TP:', TP)

1 --> SUBJECTIVE
counter/sentence_counter: 1 / 1
0 --> SUBJECTIVE
counter/sentence_counter: 1 / 2
1 --> OBJECTIVE
counter/sentence_counter: 1 / 3
1 --> SUBJECTIVE
counter/sentence_counter: 2 / 4
0 --> SUBJECTIVE
counter/sentence_counter: 2 / 5
0 --> OBJECTIVE
counter/sentence_counter: 3 / 6
0 --> OBJECTIVE
counter/sentence_counter: 4 / 7
1 --> SUBJECTIVE
counter/sentence_counter: 5 / 8
1 --> SUBJECTIVE
counter/sentence_counter: 6 / 9
1 --> SUBJECTIVE
counter/sentence_counter: 7 / 10
0 --> SUBJECTIVE
counter/sentence_counter: 7 / 11
1 --> SUBJECTIVE
counter/sentence_counter: 8 / 12
1 --> SUBJECTIVE
counter/sentence_counter: 9 / 13
1 --> SUBJECTIVE
counter/sentence_counter: 10 / 14
0 --> OBJECTIVE
counter/sentence_counter: 11 / 15
0 --> SUBJECTIVE
counter/sentence_counter: 11 / 16
1 --> OBJECTIVE
counter/sentence_counter: 11 / 17
0 --> OBJECTIVE
counter/sentence_counter: 12 / 18
0 --> SUBJECTIVE
counter/sentence_counter: 12 / 19
0 --> SUBJECTIVE
counter/sentence_counter: 12 / 20
1 --> SU

**Evaluate on bard dataset**


In [11]:
# Load the dataset
import pandas as pd

df = pd.read_csv('data/bard.csv')
df.shape

(100, 2)

In [12]:
import time

TP = 0
FP = 0
TN = 0
FN = 0

sentence_counter = 0
counter = 0

for index, row in df.iterrows():
    sentence_counter += 1
    text = row['text']
    label = row['label']
    print(text, '-->', label)

    prompt = f"""You are an expert linguist. You should decide about a sentence whether is a subjective or an objective sentence.\
    A sentence is subjective if its content is based on or influenced by personal feelings, tastes, or opinions. Otherwise,
    the sentence is objective.\
    More precisely, a sentence is subjective if one or more of the following conditions apply:\
    1. expresses an explicit personal opinion from the author (e.g., speculations to draw conclusions);\
    2. includes sarcastic or ironic expressions;\
    3. gives exhortations of personal auspices;\
    4. contains discriminating or downgrading expressions;\
    5. contains rhetorical figures that convey the author’s opinion.\
        
    Please classify the following text {text} in one of the two classes: (SUBJECTIVE, OBJECTIVE).\
    Please, answer with a single word: OBJECTIVE or SUBJECTIVE"""

    result =chatWithGPT(prompt)


    # result = chatWithAI21(prompt)
    # # chatWithAI21(prompt)
    # result = result[1:] 
    result = result.upper()
    print(label, '-->', result)
    if (label == 1 and result == 'SUBJECTIVE') or (label == 0 and result == 'OBJECTIVE'):
        counter += 1
    if (label == 0 and result == 'OBJECTIVE'):
        TN = TN +1
    if (label == 0 and result == 'SUBJECTIVE'):
        FP = FP +1
    if (label == 1 and result == 'SUBJECTIVE'):
        TP = TP +1
    if (label == 1 and result == 'OBJECTIVE'):
        FN = FN +1
    print('counter/sentence_counter:', counter, '/', sentence_counter)
    time.sleep(1)
    # if sentence_counter == 1:
    #     break

print('Accuracy:', (TP+TN)/(TP+FP+TN+FN))
print('Precision: ', TP/(TP+FP))
print('Recall: ', TP/(TP+FN))
print('F1:', TP/(TP+0.5*(FN+FP)))
print('TN:', TN)
print('FP:', FP)
print('FN:', FN)
print('TP:', TP)

The aroma of freshly baked bread always fills me with an irresistible sense of comfort and joy. --> 1
1 --> SUBJECTIVE
counter/sentence_counter: 1 / 1
The taste of a perfectly ripe strawberry is a symphony of sweetness and freshness that awakens my senses. --> 1
1 --> SUBJECTIVE
counter/sentence_counter: 2 / 2
A well-made meal cooked with love and care can be a source of genuine happiness and nourishment. --> 1
1 --> SUBJECTIVE
counter/sentence_counter: 3 / 3
The feeling of exploring a new place immersing myself in its culture and surroundings is an exhilarating adventure that leaves me forever changed. --> 1
1 --> SUBJECTIVE
counter/sentence_counter: 4 / 4
The thrill of discovering hidden gems and off-the-beaten-path destinations fills me with a sense of wonder and excitement. --> 1
1 --> SUBJECTIVE
counter/sentence_counter: 5 / 5
Traveling broadens my horizons challenges my perceptions and reminds me of the vastness and diversity of our world. --> 1
1 --> SUBJECTIVE
counter/sentence_

**Evaluate on AI21 dataset**

In [4]:
# Load the dataset
import pandas as pd

df = pd.read_csv('data/ai21.csv')
df.shape

(100, 2)

In [7]:
import time

TP = 0
FP = 0
TN = 0
FN = 0

sentence_counter = 0
counter = 0

for index, row in df.iterrows():
    sentence_counter += 1
    text = row['text']
    label = row['label']
    print(text, '-->', label)

    prompt = f"""You are an expert linguist. You should decide about a sentence whether is a subjective or an objective sentence.\
    A sentence is subjective if its content is based on or influenced by personal feelings, tastes, or opinions. Otherwise,
    the sentence is objective.\
    More precisely, a sentence is subjective if one or more of the following conditions apply:\
    1. expresses an explicit personal opinion from the author (e.g., speculations to draw conclusions);\
    2. includes sarcastic or ironic expressions;\
    3. gives exhortations of personal auspices;\
    4. contains discriminating or downgrading expressions;\
    5. contains rhetorical figures that convey the author’s opinion.\
        
    Please classify the following text {text} in one of the two classes: (SUBJECTIVE, OBJECTIVE).\
    Please, answer with a single word: OBJECTIVE or SUBJECTIVE"""

    result =chatWithGPT(prompt)


    result = result.upper()
    print(label, '-->', result)
    if (label == 1 and result == 'SUBJECTIVE') or (label == 0 and result == 'OBJECTIVE'):
        counter += 1
    if (label == 0 and result == 'OBJECTIVE'):
        TN = TN +1
    if (label == 0 and result == 'SUBJECTIVE'):
        FP = FP +1
    if (label == 1 and result == 'SUBJECTIVE'):
        TP = TP +1
    if (label == 1 and result == 'OBJECTIVE'):
        FN = FN +1
    print('counter/sentence_counter:', counter, '/', sentence_counter)
    time.sleep(1)
    # if sentence_counter == 1:
    #     break

print('Accuracy:', (TP+TN)/(TP+FP+TN+FN))
print('Precision: ', TP/(TP+FP))
print('Recall: ', TP/(TP+FN))
print('F1:', TP/(TP+0.5*(FN+FP)))
print('TN:', TN)
print('FP:', FP)
print('FN:', FN)
print('TP:', TP)

The pizza at Joe's Joint is the best in town. --> 1
1 --> SUBJECTIVE
counter/sentence_counter: 1 / 1
The new Marvel movie was a total disappointment. --> 1
1 --> SUBJECTIVE
counter/sentence_counter: 2 / 2
My dog is the cutest thing in the world. --> 1
1 --> SUBJECTIVE
counter/sentence_counter: 3 / 3
The beach is the perfect place to spend a summer day. --> 1
1 --> SUBJECTIVE
counter/sentence_counter: 4 / 4
New York City is the most exciting city I've ever visited. --> 1
1 --> SUBJECTIVE
counter/sentence_counter: 5 / 5
The rain was a welcome relief after a hot humid day. --> 1
1 --> SUBJECTIVE
counter/sentence_counter: 6 / 6
The coffee at Starbucks is overpriced and overrated. --> 1
1 --> SUBJECTIVE
counter/sentence_counter: 7 / 7
The new Taylor Swift album is a masterpiece. --> 1
1 --> SUBJECTIVE
counter/sentence_counter: 8 / 8
Camping is the best way to experience the beauty of nature. --> 1
1 --> SUBJECTIVE
counter/sentence_counter: 9 / 9
Traveling alone is the best way to learn abou